In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import random_split, DataLoader, Dataset

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data2 = pd.read_csv("data/roomPredictionData3.csv")
data2["time_int"] = data2.loc[:, "time"].astype(np.int64)
data2

,room,id,rssi,time,name,time_int
0,Office,216D23F7-E6A1-7727-A3D5-91C76258F327,-60.000,742178653.889,Office,742178653
1,Office,767B7B8B-5B66-F187-5E3F-2D91165B7273,-86.000,742178653.987,Maliha’s Room,742178653
2,Office,4B1CB61A-F138-C74E-68D6-89CBA40735F8,-89.000,742178654.006,HomePod,742178654
3,Office,AB8FA414-1C59-7DE7-F7EF-87C6CF2A303B,-78.000,742178654.012,Living room (2),742178654
4,Office,C89F2066-D85B-7A21-E292-A1735A30A295,-84.000,742178654.020,Bedroom,742178654
...,...,...,...,...,...,...
410,Office,94CE6244-F55B-5A57-E0D3-400D3E564F48,-84.000,742178712.414,NaN,742178712
411,Office,767B7B8B-5B66-F187-5E3F-2D91165B7273,-88.000,742178712.898,Maliha’s Room,742178712
412,Office,94CE6244-F55B-5A57-E0D3-400D3E564F48,-95.000,742178712.973,NaN,742178712
413,Office,94CE6244-F55B-5A57-E0D3-400D3E564F48,-93.000,742178712.984,NaN,742178712


In [3]:
data = pd.read_csv("data/roomPredictionData2.csv")
data

,room,id,rssi,time
0,Office,C89F2066-D85B-7A21-E292-A1735A30A295,-83.000,742084271.559
1,Office,216D23F7-E6A1-7727-A3D5-91C76258F327,-59.000,742084271.566
2,Office,11FE69C7-7DD2-13AF-1493-7B63E92D5B0D,-56.000,742084271.574
3,Office,DB047FFD-D211-EDF3-345E-1FE933920DC2,-83.000,742084271.574
4,Office,AB8FA414-1C59-7DE7-F7EF-87C6CF2A303B,-79.000,742084271.579
...,...,...,...,...
3274,Bedroom,94CE6244-F55B-5A57-E0D3-400D3E564F48,-76.000,742084682.740
3275,Bedroom,94CE6244-F55B-5A57-E0D3-400D3E564F48,-75.000,742084682.750
3276,Bedroom,767B7B8B-5B66-F187-5E3F-2D91165B7273,-98.000,742084682.841
3277,Bedroom,94CE6244-F55B-5A57-E0D3-400D3E564F48,-72.000,742084683.010


In [4]:
data.room.unique()

array(['Office', 'Hallway', 'Kitchen', 'Living Room', 'Bedroom'],
      dtype=object)

In [ ]:
beacon_ids = [
    "6965680D-633E-BFB7-A5FA-548907D47ADC"
    "D9DA2BE7-CF20-096C-1532-750D49104F9A"
    "79837152-C559-6F7B-337D-3AF44C58BDF6"
    "DC4C4B32-33B5-1BA3-9328-784549D963A9"
]



Below is converting the time to an arbitrary base. This will be used to group points / second to triangulate location. Each of these groups will be used as a data point to be passed through the model

In [5]:
def custom_round(x, base=1):
    return base * round(float(x)/base)

data["time_rounded"] = data["time"].apply(lambda x: custom_round(x, base=.5))

In [6]:
data.nunique()


room               5
id               237
rssi              68
time            3279
time_rounded     811
dtype: int64

For each group of rounded data, find duplicate points and compute the mean of the RSSI

In [7]:
deduped_data = data.groupby(["time_rounded","id", "room"], as_index=False)["rssi"].mean()

Re shape data in to columns as ID's and each row as the group above's RSSI values for the time frame

In [8]:
rooms = deduped_data["room"].unique()
room_labels = dict()
label_rooms = dict()

for i, room in enumerate(rooms):
    room_labels[room] = i
    label_rooms[i] = room

In [9]:
columns = np.append(["room"], deduped_data["id"].unique())
new_data = pd.DataFrame(columns=columns)

for name, group in deduped_data.groupby("time_rounded"):
    if group["room"].nunique() > 1:
        print(f"Skipping {name}")
    
    row = dict()
    row["room"] = room_labels[group["room"].unique()[0]] # Convert room to int for training
    
    for i, r in group.iterrows():
        row[r["id"]] = (r["rssi"] + 100) / 100
        
    for c in columns:
        # Set id's not found in this time as lowest value
        if c not in row:
            row[c] = 0
            
    new_data = new_data._append(
        row,
        ignore_index=True
    )
    
new_data

/var/folders/jc/85vy06cn6b13gcm4rcvz9_fw0000gq/T/ipykernel_69682/1270200350.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data = new_data._append(


,room,11FE69C7-7DD2-13AF-1493-7B63E92D5B0D,1A1B6F87-2EF2-5298-771B-4105CAB1A5D4,216D23F7-E6A1-7727-A3D5-91C76258F327,767B7B8B-5B66-F187-5E3F-2D91165B7273,A37C1D23-92BB-B847-8018-C22651E46D48,AB8FA414-1C59-7DE7-F7EF-87C6CF2A303B,C89F2066-D85B-7A21-E292-A1735A30A295,DB047FFD-D211-EDF3-345E-1FE933920DC2,E834AAA0-5049-34E4-BFCA-2D38C282CC37,...,9AB58CB2-475D-7F24-F4F2-437580A26C36,F2E974F8-CF56-72D1-0752-65B159B83A6B,C8903AD9-3109-B6A7-1057-DF90EFAD7B0F,D45DF8DD-5F77-30FC-3638-E8D88316C386,0AD1657E-1A49-D3CF-9DC8-15EAA4752FBA,7A1BE164-7913-7278-169F-18F4E3ABE1C6,4E0F5991-CA35-5715-16CE-3E5B41A432A0,5FA58859-AB20-E90C-5D7D-4E2BDC9FA455,62279955-64BA-D757-8022-0B57D48FA230,7212A976-AD05-83D9-C3B5-9D5D972278FA
0,0.000,0.440,0.190,0.410,0.200,0.390,0.210,0.170,0.170,0.150,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.150,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.400,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.090,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
807,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
808,0.000,0.000,0.000,0.000,0.110,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
809,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [10]:
new_data

,room,11FE69C7-7DD2-13AF-1493-7B63E92D5B0D,1A1B6F87-2EF2-5298-771B-4105CAB1A5D4,216D23F7-E6A1-7727-A3D5-91C76258F327,767B7B8B-5B66-F187-5E3F-2D91165B7273,A37C1D23-92BB-B847-8018-C22651E46D48,AB8FA414-1C59-7DE7-F7EF-87C6CF2A303B,C89F2066-D85B-7A21-E292-A1735A30A295,DB047FFD-D211-EDF3-345E-1FE933920DC2,E834AAA0-5049-34E4-BFCA-2D38C282CC37,...,9AB58CB2-475D-7F24-F4F2-437580A26C36,F2E974F8-CF56-72D1-0752-65B159B83A6B,C8903AD9-3109-B6A7-1057-DF90EFAD7B0F,D45DF8DD-5F77-30FC-3638-E8D88316C386,0AD1657E-1A49-D3CF-9DC8-15EAA4752FBA,7A1BE164-7913-7278-169F-18F4E3ABE1C6,4E0F5991-CA35-5715-16CE-3E5B41A432A0,5FA58859-AB20-E90C-5D7D-4E2BDC9FA455,62279955-64BA-D757-8022-0B57D48FA230,7212A976-AD05-83D9-C3B5-9D5D972278FA
0,0.000,0.440,0.190,0.410,0.200,0.390,0.210,0.170,0.170,0.150,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.150,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.400,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.090,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
807,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
808,0.000,0.000,0.000,0.000,0.110,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
809,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


# Base Class

https://towardsdatascience.com/from-pytorch-to-pytorch-lightning-a-gentle-introduction-b371b7caaf09

https://lightning.ai/docs/torchmetrics/stable//index.html
Would replace Trainer from scratch 


Also install tensorboard for viewing experiments


In [1]:
from scripts.shared_utilities import *

data_module = RoomPredictorDataModule("roomPredictionData2.csv", [.9, .05, .05], 8, 9, "data")

data_module.prepare_data()
data_module.setup("test")

input: torch.Tensor = data_module.train_dataloader().dataset[0]

input[0].size()

ModuleNotFoundError: No module named 'scripts'

# Training

In [19]:
from lightning.pytorch.callbacks import ModelSummary, EarlyStopping, LearningRateFinder

num_epochs = 100
num_hidden = np.mean(np.array([len(columns), len(rooms)])).astype(np.int8)
lr = 1e-4
batch_size = 8
seed = 1536282

model_summary = ModelSummary(max_depth=-1)
early_stopping = EarlyStopping(monitor="val_loss", mode="min")
learning_rate = LearningRateFinder()

trainer = L.Trainer(
    max_epochs=num_epochs, 
    callbacks=[model_summary, early_stopping],
    profiler="simple")

model = Classifier(num_hidden, 5, lr)
data_module = RoomPredictorDataModule(file, ratios=[.8,.1,.1], batch_size=batch_size, target_column="room")

trainer.fit(model=model, datamodule=data_module)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/RoomPredictor/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 0      | train
1 | model.0 | LazyLinear       | 0      | train
2 | model.1 | ReLU             | 0      | train
3 | model.2 | Dropout          | 0      | train
4 | model.3 | LazyLinear       | 0      | train
5 | loss    | CrossEntropyLoss | 0      | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/RoomPredictor/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/opt/homebrew/Caskroom/miniforge/base/envs/RoomPredictor/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                 

In [1]:
import coremltools as ct
from scripts.shared_utilities import *

In [11]:
model = Classifier.load_from_checkpoint("logs/classifier/version_13/checkpoints/epoch=971-step=99144.ckpt")

/opt/homebrew/Caskroom/miniforge/base/envs/RoomPredictor/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
model.eval()

Classifier(
  (model): Sequential(
    (0): LazyLinear(in_features=0, out_features=180, bias=True)
    (1): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
    (2): SELU()
    (3): Dropout(p=0.3, inplace=False)
    (4): LazyLinear(in_features=0, out_features=5, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [15]:
import torch

model.eval()

example_input = torch.rand(1, model.hparams.num_inputs, model.hparams.num_features) 
traced_model = model.to_torchscript(method="trace", example_inputs=example_input)



In [18]:
model_from_trace = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=example_input.shape)],
)
model_from_trace.save("newmodel_from_trace.mlpackage")

When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 5309.81 passes/s]


In [10]:
example_input

tensor([[[0.4963, 0.7682, 0.0885,  ..., 0.8059, 0.1839, 0.7243],
         [0.1466, 0.2881, 0.6471,  ..., 0.9932, 0.7875, 0.4437],
         [0.6753, 0.0095, 0.0729,  ..., 0.2614, 0.8203, 0.4365],
         ...,
         [0.0932, 0.4086, 0.8896,  ..., 0.5894, 0.5028, 0.1075],
         [0.2787, 0.2665, 0.0046,  ..., 0.5407, 0.2489, 0.7414],
         [0.5872, 0.0623, 0.1760,  ..., 0.4622, 0.0103, 0.3348]]])